In [2]:
#importing the libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
from numpy import genfromtxt
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

/home/amerelsamman/anaconda3/envs/dlchem/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch.utils.data import Dataset, DataLoader
class embeddings(Dataset):
  def __init__(self,x_data,y_data):
    self.x_data = torch.tensor(x_data,dtype=torch.float32)
    self.y_data = torch.tensor(y_data,dtype=torch.float32)
    self.length = self.x_data.shape[0]
  def __getitem__(self,idx):
    return self.x_data[idx],self.y_data[idx]
  def __len__(self):
    return self.length


In [4]:
#creating the network
from torch import nn
class net(nn.Module):
  def __init__(self,input_size,output_size):
    super(net,self).__init__()
    print(output_size)
    self.l1 = nn.Linear(input_size,70)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(70,output_size)
  def forward(self,x_data):
    output = self.l1(x_data)
    output = self.relu(output)
    output = self.l2(output)
    return output




In [9]:
# Load data into a Pandas DataFrame
XY = genfromtxt('../data/embs/embsdensity-ince/embs250-5000woccspca.csv',delimiter=',',encoding='utf-8-sig',skip_header=1)
# Split data into X and y
x_data = XY[:,0:128]
y_data = XY[:,138:141]
label = XY[:,133:136]

model = net(x_data.shape[1],y_data.shape[1])
criterion = nn.MSELoss() #type of loss function
optimizer = torch.optim.SGD(model.parameters(),lr=0.001) #learning rate
epochs = 1500 #complexity of models

dataset = embeddings(x_data,y_data)

dataloader = DataLoader(dataset=dataset,shuffle=True,batch_size=100)

costval = []
X_train, X_test, y_train, y_test, label_train, label_test = train_test_split(x_data, y_data, label, test_size=0.25, random_state= 50)

# Standardize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = torch.tensor(scaler_X.fit_transform(X_train),dtype=torch.float32)
X_test_scaled = torch.tensor(scaler_X.transform(X_test),dtype=torch.float32)

y_train_scaled = torch.tensor(scaler_y.fit_transform(y_train),dtype=torch.float32)
y_test_scaled = torch.tensor(scaler_y.transform(y_test),dtype=torch.float32)

for j in range(epochs):
  for i,(x_train,y_train) in enumerate(dataloader):
    #prediction
    y_pred_scaled = model(X_train_scaled)
    #calculating loss
    cost = criterion(y_pred_scaled,y_train_scaled)
    #backprop
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
  if j%50 == 0:
    print(cost)
    costval.append(cost)
  

3
tensor(1.0549, grad_fn=<MseLossBackward0>)
tensor(0.4109, grad_fn=<MseLossBackward0>)
tensor(0.1111, grad_fn=<MseLossBackward0>)
tensor(0.0710, grad_fn=<MseLossBackward0>)
tensor(0.0578, grad_fn=<MseLossBackward0>)
tensor(0.0499, grad_fn=<MseLossBackward0>)
tensor(0.0443, grad_fn=<MseLossBackward0>)
tensor(0.0401, grad_fn=<MseLossBackward0>)
tensor(0.0368, grad_fn=<MseLossBackward0>)
tensor(0.0340, grad_fn=<MseLossBackward0>)
tensor(0.0317, grad_fn=<MseLossBackward0>)
tensor(0.0298, grad_fn=<MseLossBackward0>)
tensor(0.0282, grad_fn=<MseLossBackward0>)
tensor(0.0267, grad_fn=<MseLossBackward0>)
tensor(0.0254, grad_fn=<MseLossBackward0>)
tensor(0.0243, grad_fn=<MseLossBackward0>)
tensor(0.0233, grad_fn=<MseLossBackward0>)
tensor(0.0224, grad_fn=<MseLossBackward0>)
tensor(0.0216, grad_fn=<MseLossBackward0>)
tensor(0.0208, grad_fn=<MseLossBackward0>)
tensor(0.0201, grad_fn=<MseLossBackward0>)
tensor(0.0195, grad_fn=<MseLossBackward0>)
tensor(0.0189, grad_fn=<MseLossBackward0>)
tensor(0.

In [15]:

#prediction
y_pred_scaled = model(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled.detach().numpy())

#print(y_test)
#y_test = torch.tensor(y_test,dtype=torch.float32)
#calculating loss
cost = criterion(torch.tensor(y_pred),torch.tensor(y_test))

print(cost)

tensor(8.9956e-06, dtype=torch.float64)


In [14]:
import numpy as np

print(np.shape(label_test))
print(np.shape(y_pred))
use_train_for_test = False

for i in range(np.shape(y_pred)[1]):

    if use_train_for_test == True: 
        save_predictions = np.column_stack((y_pred[:,i],y_train[:,i],label_train))
    elif use_train_for_test == False: 
        save_predictions = np.column_stack((y_pred[:,i],y_test[:,i],label_test))
        
    save_pred_filepath = '../data/embs/embsdensity-ince/nntestpredictions%s.csv'  %(str(i)) 
    np.savetxt(save_pred_filepath,save_predictions,delimiter=',')


(1415, 3)
(1415, 3)
